In [ ]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Define constants
ZIP_FILE = "archive.zip"  # Path to your zip file
EXTRACTION_DIR = "./extracted_data"
IMAGE_SIZE = (64, 64)  # Image dimensions (height, width)
BATCH_SIZE = 32
EPOCHS = 10

# Step 1: Extract the ZIP file
def extract_zip_file(zip_path, extraction_directory):
    if not os.path.exists(extraction_directory):
        os.makedirs(extraction_directory)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_directory)
    print(f"Files extracted to: {extraction_directory}")

# Step 2: Load and preprocess the OHLC data
def load_ohlc_data(extraction_dir):
    """Load OHLC data and preprocess it for candlestick image creation."""
    csv_files = glob.glob(os.path.join(extraction_dir, "*.csv"))
    if not csv_files:
        raise FileNotFoundError("No CSV file found in the extraction directory.")

    ohlc = pd.read_csv(csv_files[0])
    # Ensure columns have correct formatting
    ohlc = ohlc[['open', 'high', 'low', 'close']].dropna().reset_index(drop=True)
    print("OHLC data loaded successfully.")
    return ohlc

# Step 3: Create candlestick images from OHLC data
def create_candlestick_images(ohlc, save_dir, image_size=(64, 64)):
    """Create images of 3-candle formations and save them to disk."""
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    labels = []
    for i in range(len(ohlc) - 3):
        # Extract three consecutive candlesticks
        data = ohlc.iloc[i:i+3]

        # Create a blank canvas
        canvas = np.zeros((image_size[0], image_size[1], 3), dtype=np.uint8)

        # Scale OHLC prices to fit within the canvas
        min_price = data[['open', 'high', 'low', 'close']].min().min()
        max_price = data[['open', 'high', 'low', 'close']].max().max()
        scale = image_size[1] / (max_price - min_price)

        # Draw candlesticks
        for j, row in enumerate(data.itertuples()):
            x = int((j + 1) * (image_size[0] / 4))  # Position on the x-axis
            open_price = int((row.open - min_price) * scale)
            close_price = int((row.close - min_price) * scale)
            high_price = int((row.high - min_price) * scale)
            low_price = int((row.low - min_price) * scale)

            color = (0, 255, 0) if row.close >= row.open else (255, 0, 0)  # Green for bullish, red for bearish
            cv2.line(canvas, (x, image_size[1] - low_price), (x, image_size[1] - high_price), (255, 255, 255), 1)
            cv2.rectangle(canvas, (x - 3, image_size[1] - open_price), (x + 3, image_size[1] - close_price), color, -1)

        # Save image
        image_path = os.path.join(save_dir, f"candlestick_{i}.png")
        cv2.imwrite(image_path, canvas)

        # Determine label (1 for bullish, 0 for bearish) based on the next candlestick
        next_close = ohlc.iloc[i+3]['close']
        current_close = ohlc.iloc[i+2]['close']
        labels.append(1 if next_close > current_close else 0)

    print(f"Candlestick images saved to: {save_dir}")
    return labels

# Step 4: Load and preprocess image data for CNN
def load_image_data(image_dir, labels, image_size=(64, 64)):
    """Load image data and associate it with corresponding labels."""
    images = []
    for image_path in sorted(glob.glob(os.path.join(image_dir, "*.png"))):
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, image_size)
        images.append(image)

    images = np.array(images).reshape(-1, image_size[0], image_size[1], 1)  # Reshape for CNN
    labels = to_categorical(labels, 2)  # Convert labels to one-hot encoding (binary classification)

    # Normalize image pixel values
    images = images / 255.0

    print(f"Loaded {len(images)} images and labels.")
    return images, labels

# Step 5: Train the CNN model
def train_image_model(X, y, epochs=10, batch_size=32):
    """Train a CNN model using the preprocessed image data."""
    # Split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Augment data with ImageDataGenerator
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
    datagen.fit(X_train)

    # Train the model
    history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                        validation_data=(X_test, y_test), epochs=epochs)

    return model, history

# Main function
def main():
    # Step 1: Extract ZIP file
    extract_zip_file(ZIP_FILE, EXTRACTION_DIR)

    # Step 2: Load and preprocess OHLC data
    ohlc = load_ohlc_data(EXTRACTION_DIR)

    # Step 3: Create candlestick images
    image_dir = os.path.join(EXTRACTION_DIR, "candlestick_images")
    labels = create_candlestick_images(ohlc, image_dir, IMAGE_SIZE)

    # Step 4: Load image data
    images, labels = load_image_data(image_dir, labels, IMAGE_SIZE)

    # Step 5: Train image model
    model, history = train_image_model(images, labels, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # Save the trained model
    model.save("candlestick_image_model.h5")
    print("Model saved successfully.")

if __name__ == "__main__":
    main()


Files extracted to: ./extracted_data
OHLC data loaded successfully.
Candlestick images saved to: ./extracted_data/candlestick_images
Loaded 116664 images and labels.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 548s 187ms/step - accuracy: 0.5124 - loss: 0.6934 - val_accuracy: 0.5131 - val_loss: 0.6929
Epoch 2/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 568s 189ms/step - accuracy: 0.5126 - loss: 0.6929 - val_accuracy: 0.5131 - val_loss: 0.6929
Epoch 3/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 562s 189ms/step - accuracy: 0.5121 - loss: 0.6929 - val_accuracy: 0.5131 - val_loss: 0.6931
Epoch 4/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 565s 190ms/step - accuracy: 0.5120 - loss: 0.6930 - val_accuracy: 0.5131 - val_loss: 0.6929
Epoch 5/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 550s 189ms/step - accuracy: 0.5081 - loss: 0.6930 - val_accuracy: 0.5131 - val_loss: 0.6928
Epoch 6/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 552s 189ms/step - accuracy: 0.5100 - loss: 0.6930 - val_accuracy: 0.5131 - val_loss: 0.6928
Epoch 7/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 544s 187ms/step - accuracy: 0.5072 - loss: 0.6931 - val_accuracy: 0.5131 - val_loss: 0.6928
Epoch 8/10
2917/2917 ━━━━━━━━━━━━━━━━━━━━ 552s 189ms/step - ac

Model saved successfully.


In [ ]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# ---- Constants ----
ZIP_FILE = "archive.zip"  # Replace with your dataset zip file
EXTRACTION_DIR = "./extracted_data"
IMAGE_SIZE = (64, 64)  # Image dimensions
BATCH_SIZE = 32
EPOCHS = 10
MAX_IMAGES = 100000  # Optional: Limit the number of generated images for large datasets


# ---- Step 1: Extract ZIP File ----
def extract_zip_file(zip_path, extraction_dir):
    """Extract a ZIP file to a specific directory."""
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_dir)
    print(f"Files extracted to: {extraction_dir}")


# ---- Step 2: Load and Preprocess OHLC Data ----
def load_ohlc_data(folder_path):
    """Load OHLC data from extracted CSV files."""
    csv_file = glob.glob(os.path.join(folder_path, "*.csv"))[0]  # Grab the first CSV file
    if not csv_file:
        raise FileNotFoundError("No OHLC data file found in the directory!")
    ohlc = pd.read_csv(csv_file)
    ohlc = ohlc[['open', 'high', 'low', 'close']].dropna().reset_index(drop=True)
    print("Loaded OHLC data successfully.")
    return ohlc


# ---- Step 3: Create Candlestick Images in Chunks ----
def create_candlestick_images(ohlc, output_dir, image_size=(64, 64), batch_size=1000):
    """Convert candlestick formations into images and generate corresponding labels in batches."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    labels = []
    image_count = 0

    # Ensure we stop when there are less than 4 candlesticks left (for labels)
    for batch_start in range(0, min(len(ohlc) - 3, MAX_IMAGES), batch_size):
        batch_end = min(batch_start + batch_size, len(ohlc) - 3)  # Adjust for last batch
        batch_labels = []

        for i in range(batch_start, batch_end):
            # Extract three consecutive candlesticks
            three_candles = ohlc.iloc[i:i + 3]
            next_close = ohlc.iloc[i + 3]['close']  # Compare next candlestick's close
            last_close = ohlc.iloc[i + 2]['close']

            # Determine the label
            batch_labels.append(1 if next_close > last_close else 0)

            # Create a candlestick image canvas
            canvas = np.zeros((*image_size, 3), dtype=np.uint8)
            min_price = three_candles.min().min()
            max_price = three_candles.max().max()
            scale = (image_size[1] / (max_price - min_price))

            # Draw candlesticks
            for j, row in enumerate(three_candles.itertuples()):
                x = int((j + 1) * (image_size[0] / 4))
                open_price = int((row.open - min_price) * scale)
                close_price = int((row.close - min_price) * scale)
                high_price = int((row.high - min_price) * scale)
                low_price = int((row.low - min_price) * scale)

                # Green if bullish, red if bearish
                color = (0, 255, 0) if row.close >= row.open else (255, 0, 0)
                cv2.line(canvas, (x, image_size[1] - low_price), (x, image_size[1] - high_price), (255, 255, 255), 1)
                cv2.rectangle(canvas, (x - 3, image_size[1] - open_price), (x + 3, image_size[1] - close_price), color, -1)

            # Save the image
            image_path = os.path.join(output_dir, f"candle_{image_count}.png")
            cv2.imwrite(image_path, canvas)
            image_count += 1

        labels.extend(batch_labels)
        print(f"Batch processed: {batch_start}-{batch_end}, Total images: {image_count}")

    print(f"Total candlestick images created: {image_count}")
    return labels


# ---- Step 4: Data Generator for Training ----
def data_generator(image_dir, labels, image_size=(64, 64), batch_size=32):
    """Yield batches of images and labels without exceeding the labels list."""
    images = sorted(glob.glob(os.path.join(image_dir, "*.png")))
    num_samples = len(labels)  # Match the number of labels explicitly

    while True:
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_images = []
            batch_labels = []

            for idx in range(start_idx, end_idx):
                # Ensure idx does not exceed the number of images or labels
                if idx < len(images) and idx < len(labels):
                    image = cv2.imread(images[idx], cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, image_size)
                    batch_images.append(image)
                    batch_labels.append(labels[idx])
                else:
                    # Handle cases where idx exceeds the length of images or labels
                    break

            if len(batch_images) == 0:
                break  # Stop if no images are left

            # Format batches for CNN
            batch_images = np.array(batch_images).reshape(-1, image_size[0], image_size[1], 1) / 255.0
            batch_labels = to_categorical(batch_labels, num_classes=2)

            yield batch_images, batch_labels


# ---- Step 5: Train CNN Model ----
def train_image_model(image_dir, labels, image_size=(64, 64), epochs=10, batch_size=32):
    """Train the CNN model using a data generator."""
    # Split data into training and validation sets
    num_samples = len(labels)
    train_size = int(0.8 * num_samples)

    train_generator = data_generator(image_dir, labels[:train_size], image_size, batch_size)
    val_generator = data_generator(image_dir, labels[train_size:], image_size, batch_size)

    train_steps = train_size // batch_size
    val_steps = (num_samples - train_size) // batch_size

    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(2, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model using the generated data
    model.fit(
        train_generator,
        steps_per_epoch=train_steps,
        validation_data=val_generator,
        validation_steps=val_steps,
        epochs=epochs
    )

    print("Model training completed!")
    return model


# ---- Step 6: Main Function ----
def main():
    # Step 1: Extract the zip file
    extract_zip_file(ZIP_FILE, EXTRACTION_DIR)

    # Step 2: Load OHLC data
    ohlc = load_ohlc_data(EXTRACTION_DIR)

    # Step 3: Generate candlestick images and labels
    image_dir = os.path.join(EXTRACTION_DIR, "candlestick_images")
    labels = create_candlestick_images(ohlc, image_dir, IMAGE_SIZE)

    # Step 4: Train the CNN model
    model = train_image_model(image_dir, labels, image_size=IMAGE_SIZE, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # Step 5: Save the model
    model.save("candlestick_cnn_model.h5")
    print("CNN model saved successfully!")


# ---- Run the Code ----
if __name__ == "__main__":
    main()


Files extracted to: ./extracted_data
Loaded OHLC data successfully.
Batch processed: 0-1000, Total images: 1000
Batch processed: 1000-2000, Total images: 2000
Batch processed: 2000-3000, Total images: 3000
Batch processed: 3000-4000, Total images: 4000
Batch processed: 4000-5000, Total images: 5000
Batch processed: 5000-6000, Total images: 6000
Batch processed: 6000-7000, Total images: 7000
Batch processed: 7000-8000, Total images: 8000
Batch processed: 8000-9000, Total images: 9000
Batch processed: 9000-10000, Total images: 10000
Batch processed: 10000-11000, Total images: 11000
Batch processed: 11000-12000, Total images: 12000
Batch processed: 12000-13000, Total images: 13000
Batch processed: 13000-14000, Total images: 14000
Batch processed: 14000-15000, Total images: 15000
Batch processed: 15000-16000, Total images: 16000
Batch processed: 16000-17000, Total images: 17000
Batch processed: 17000-18000, Total images: 18000
Batch processed: 18000-19000, Total images: 19000
Batch process

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 446s 177ms/step - accuracy: 0.5074 - loss: 0.6936 - val_accuracy: 0.5167 - val_loss: 0.6927
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 442s 177ms/step - accuracy: 0.5111 - loss: 0.6930 - val_accuracy: 0.5166 - val_loss: 0.6927
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 442s 177ms/step - accuracy: 0.5109 - loss: 0.6930 - val_accuracy: 0.5164 - val_loss: 0.6927
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 435s 174ms/step - accuracy: 0.5110 - loss: 0.6930 - val_accuracy: 0.5164 - val_loss: 0.6927
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 442s 177ms/step - accuracy: 0.5110 - loss: 0.6929 - val_accuracy: 0.5163 - val_loss: 0.6927
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 425s 170ms/step - accuracy: 0.5110 - loss: 0.6929 - val_accuracy: 0.5163 - val_loss: 0.6927
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 444s 177ms/step - accuracy: 0.5110 - loss: 0.6929 - val_accuracy: 0.5165 - val_loss: 0.6927
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 428s 171ms/step - ac

Model training completed!
CNN model saved successfully!
